In [1]:
import pandas as pd

raw_erwann_history = "data/erwann_full.csv"
raw_irfan_history = "data/irfan.csv"
raw_kaggle_data = "data/urldata.csv"

unique_irfan_history = "data/unique_irfan.csv"
unique_erwann_history = "data/unique_erwann_history"

kaggle_set_folder = "data/kaggle_set_folder"
kaggle_test_set_folder = "data/kaggle_test_set_folder"

small_kaggle_set_folder = "data/small_kaggle_set_folder"
small_kaggle_test_set_folder = "data/small_kaggle_test_set_folder"

history_set_folder = "data/history_set_folder"
history_test_set_folder = "data/history_test_set_folder"

combine_set_folder = "data/combined_set_folder"
combine_test_set_folder = "data/combined_test_set_folder"

valid_folder_name = "valid"
malicious_folder_name = "malicious"

SEED = 42
PADDING_SIZE = 75
HISTORY_TRAIN_RATION = 0.9

pad = ""
for i in range(PADDING_SIZE):
    pad = pad + " "

In [2]:
kaggle_data = pd.read_csv(raw_kaggle_data)
kaggle_data = kaggle_data.drop('Unnamed: 0', axis=1)
kaggle_data = kaggle_data.drop('label', axis=1)

kaggle_data = kaggle_data.sample(frac=1, random_state=SEED).reset_index(drop=True)

#Pad Kaggle_dataset
kaggle_data = kaggle_data.copy()
kaggle_data["url"] = kaggle_data["url"].str.pad(width=PADDING_SIZE, side='right', fillchar=' ')
for i in range(len(kaggle_data)):
    if i < 65000:
        if len(kaggle_data["url"][i]) > PADDING_SIZE:
            kaggle_data["url"][i] = kaggle_data["url"][i][:PADDING_SIZE]
        if i % 10000 == 0:
            print(str(i) + " done")
print("KAGGLE SET PADDING DONE")

#Create small_kaggle_dataset
i = 0
valid = 0
malicious = 0
test = 0
malicious_test = 0
valid_test = 0
for index, row in kaggle_data.iterrows():
    if malicious > 10000 and valid > 50000 and test > 1200:
        break
    elif malicious > 10000 and valid > 50000 and test < 1200:
        if row["result"] == 1:
            if malicious_test > 200:
                pass
            else:
                with open(f"{small_kaggle_test_set_folder}/{malicious_folder_name}/{malicious_test}.txt", "w") as f:
                    f.write(row["url"].replace("", " ")[1: -1])
                malicious_test = malicious_test + 1
        else:
            if valid_test > 1000:
                pass
            else:
                with open(f"{small_kaggle_test_set_folder}/{valid_folder_name}/{valid_test}.txt", "w") as f:
                    f.write(row["url"].replace("", " ")[1: -1])
                valid_test = valid_test + 1
    else:
        if row["result"] == 1:
            if malicious > 10000:
                pass
            else:
                with open(f"{small_kaggle_set_folder}/{malicious_folder_name}/{malicious}.txt", "w") as f:
                    f.write(row["url"].replace("", " ")[1: -1])
                malicious = malicious + 1
        else:
            if valid > 50000:
                pass
            else:
                with open(f"{small_kaggle_set_folder}/{valid_folder_name}/{valid}.txt", "w") as f:
                    f.write(row["url"].replace("", " ")[1: -1])
                valid = valid + 1
    i = i + 1
    if i % 100000 == 0:
        print(str(i) + " done")
print("KAGGLE SMALL DATASET DONE")

0 done


<ipython-input-2-53286fa91900>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kaggle_data["url"][i] = kaggle_data["url"][i][:PADDING_SIZE]


10000 done
20000 done
30000 done
40000 done
50000 done
60000 done
KAGGLE SET PADDING DONE
100000 done
200000 done
300000 done
400000 done
KAGGLE SMALL DATASET DONE


In [3]:
kaggle_data = pd.read_csv(raw_kaggle_data)
kaggle_data = kaggle_data.drop('Unnamed: 0', axis=1)
kaggle_data = kaggle_data.drop('label', axis=1)

kaggle_data = kaggle_data.sample(frac=1, random_state=SEED).reset_index(drop=True)

TOTAL_KAGGLE_VALID = (kaggle_data.result == 0).sum()
TOTAL_KAGGLE_MALICIOUS = (kaggle_data.result == 1).sum()

#Pad Kaggle_dataset
kaggle_data = kaggle_data.copy()
kaggle_data["url"] = kaggle_data["url"].str.pad(width=PADDING_SIZE, side='right', fillchar=' ')
for i in range(len(kaggle_data)):
    if len(kaggle_data["url"][i]) > PADDING_SIZE:
        kaggle_data["url"][i] = kaggle_data["url"][i][:PADDING_SIZE]
    if i % 30000 == 0:
        print(str(i) + " done")
print("KAGGLE SET PADDING DONE")

#Create kaggle_dataset
i = 0
valid = 0
malicious = 0
test = 0
malicious_test = 0
valid_test = 0
for index, row in kaggle_data.iterrows():
    if valid > int(HISTORY_TRAIN_RATION * TOTAL_KAGGLE_VALID) and malicious > int(
            HISTORY_TRAIN_RATION * TOTAL_KAGGLE_MALICIOUS) and test > int(
        HISTORY_TRAIN_RATION * len(kaggle_data.index)):
        break
    elif valid > int(HISTORY_TRAIN_RATION * TOTAL_KAGGLE_VALID) and malicious > int(
            HISTORY_TRAIN_RATION * TOTAL_KAGGLE_MALICIOUS) and test < int(
        HISTORY_TRAIN_RATION * len(kaggle_data.index)):
        if row["result"] == 1:
            with open(f"{kaggle_test_set_folder}/{malicious_folder_name}/{malicious_test}.txt", "w") as f:
                f.write(row["url"].replace("", " ")[1: -1])
            malicious_test = malicious_test + 1
            test = test + 1
        else:
            with open(f"{kaggle_test_set_folder}/{valid_folder_name}/{valid_test}.txt", "w") as f:
                f.write(row["url"].replace("", " ")[1: -1])
            valid_test = valid_test + 1
            test = test + 1
    else:
        if row["result"] == 1:
            with open(f"{kaggle_set_folder}/{malicious_folder_name}/{malicious}.txt", "w") as f:
                f.write(row["url"].replace("", " ")[1: -1])
            malicious = malicious + 1
        else:
            with open(f"{kaggle_set_folder}/{valid_folder_name}/{valid}.txt", "w") as f:
                f.write(row["url"].replace("", " ")[1: -1])
            valid = valid + 1
    i = i + 1
    if i % 100000 == 0:
        print(str(i) + " done")
print("KAGGLE DATASET DONE")

0 done


<ipython-input-3-5476516a8004>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kaggle_data["url"][i] = kaggle_data["url"][i][:PADDING_SIZE]


30000 done
60000 done
90000 done
120000 done
150000 done
180000 done
210000 done
240000 done
270000 done
300000 done
330000 done
360000 done
390000 done
420000 done
450000 done
KAGGLE SET PADDING DONE
100000 done
200000 done
300000 done
400000 done
KAGGLE DATASET DONE


In [4]:
#Make unique url cvs history
with open('data/erwann_full.csv', 'r') as in_file, open(unique_erwann_history, 'w') as out_file:
    seen = set()  # set for fast O(1) amortized lookup
    for line in in_file:
        if line in seen: continue  # skip duplicate

        seen.add(line)
        out_file.write(line)

with open('data/irfan.csv', 'r') as in_file, open(unique_irfan_history, 'w') as out_file:
    seen = set()  # set for fast O(1) amortized lookup
    for line in in_file:
        if line in seen: continue  # skip duplicate

        seen.add(line)
        out_file.write(line)
print("UNIQUE URL DONE")

#Load Unique history set
erwann_data = pd.read_csv(unique_erwann_history)
irfan_data = pd.read_csv(unique_irfan_history)

#Pad history_dataset
erwann_data = erwann_data.copy()
erwann_data["url"] = erwann_data["url"].str.pad(width=PADDING_SIZE, side='right', fillchar=' ')
for i in range(len(erwann_data)):
    if len(erwann_data["url"][i]) > PADDING_SIZE:
        erwann_data["url"][i] = erwann_data["url"][i][:PADDING_SIZE]

irfan_data = irfan_data.copy()
irfan_data["url"] = irfan_data["url"].str.pad(width=PADDING_SIZE, side='right', fillchar=' ')
for i in range(len(irfan_data)):
    if len(irfan_data["url"][i]) > PADDING_SIZE:
        irfan_data["url"][i] = irfan_data["url"][i][:PADDING_SIZE]
print("HISTORY SET PADDING DONE")

#create history_set_folder
valid = 0
valid_test = 0
temp_valid = 0
for index, row in erwann_data.iterrows():
    if temp_valid < int(HISTORY_TRAIN_RATION * len(erwann_data.index)):
        with open(f"{history_set_folder}/{valid_folder_name}/{valid}.txt", "w") as f:
            f.write(row["url"].replace("", " ")[1: -1])
        valid = valid + 1
        temp_valid = temp_valid + 1
    else:
        with open(f"{history_test_set_folder}/{valid_folder_name}/{valid_test}.txt", "w") as f:
            f.write(row["url"].replace("", " ")[1: -1])
        valid_test = valid_test + 1
print("ERWANN DONE")

temp_valid = 0
for index, row in irfan_data.iterrows():
    if temp_valid < int(HISTORY_TRAIN_RATION * len(irfan_data.index)):
        with open(f"{history_set_folder}/{valid_folder_name}/{valid}.txt", "w") as f:
            f.write(row["url"].replace("", " ")[1: -1])
        valid = valid + 1
        temp_valid = temp_valid + 1
    else:
        with open(f"{history_test_set_folder}/{valid_folder_name}/{valid_test}.txt", "w") as f:
            f.write(row["url"].replace("", " ")[1: -1])
        valid_test = valid_test + 1
print("IRFAN DONE")
print("HISTORY SMALL DATASET DONE")

UNIQUE URL DONE
HISTORY SET PADDING DONE
ERWANN DONE
IRFAN DONE
HISTORY SMALL DATASET DONE


In [5]:
#combined dataset
import os
import shutil

valid_file_index = 0
malicious_file_index = 0

#history train valid
for file in os.listdir(f"{history_set_folder}/valid"):
    if file.endswith(".txt"):
        shutil.copy(f"{history_set_folder}/valid/{file}", f"{combine_set_folder}/valid/{valid_file_index}.txt")
        valid_file_index = valid_file_index + 1
#history test valid
for file in os.listdir(f"{history_test_set_folder}/valid"):
    if file.endswith(".txt"):
        shutil.copy(f"{history_test_set_folder}/valid/{file}", f"{combine_test_set_folder}/valid/{valid_file_index}.txt")
        valid_file_index = valid_file_index + 1
#small train valid
for file in os.listdir(f"{small_kaggle_set_folder}/valid"):
    if file.endswith(".txt"):
        shutil.copy(f"{small_kaggle_set_folder}/valid/{file}", f"{combine_set_folder}/valid/{valid_file_index}.txt")
        valid_file_index = valid_file_index + 1
#small test valid
for file in os.listdir(f"{small_kaggle_test_set_folder}/valid"):
    if file.endswith(".txt"):
        shutil.copy(f"{small_kaggle_test_set_folder}/valid/{file}", f"{combine_test_set_folder}/valid/{valid_file_index}.txt")
        valid_file_index = valid_file_index + 1
#small train malicious
for file in os.listdir(f"{small_kaggle_set_folder}/malicious"):
    if file.endswith(".txt"):
        shutil.copy(f"{small_kaggle_set_folder}/malicious/{file}", f"{combine_set_folder}/malicious/{malicious_file_index}.txt")
        malicious_file_index = malicious_file_index + 1
#small test malicious
for file in os.listdir(f"{small_kaggle_test_set_folder}/malicious"):
    if file.endswith(".txt"):
        shutil.copy(f"{small_kaggle_test_set_folder}/malicious/{file}", f"{combine_test_set_folder}/malicious/{malicious_file_index}.txt")
        malicious_file_index = malicious_file_index + 1
print("COMBINED TRAIN AND TEST SETS DONE")

COMBINED TRAIN AND TEST SETS DONE
